# Maximizing the Single-Step Entropy with Unknowns

The single-step entropy is given by the following formula.

$$  H[P] = - \sum_{i=1}^n p_i \ln p_i $$

If we know what the total sum of the values must be (say, 1.0 for strict probabilities) then we can immediately obtain the answer.  We just subtract the knowns from 1.0 and divide by the number of unknowns; there is really no freedom to choose.  On the other hand, if we assume the values are *relative weights*, then we have the additional constraint that the entropy must be maximized when normalized, so we must divide by the sum of the values.

If instead of simple probabilities we have relative weights $W = \{ w_1, w_2, \ldots, w_n \}$, then we must divide by the sum of the weights to obtain probabilities.  Letting $T = \sum_{i=1}^n w_i$ be the sum of all the weights, we can express the entropy of this distribution as follows.

$$  H[W] = - \sum_{i=1}^n \frac{w_i}{T} \ln \frac{w_i}{T}  $$

Given a set of weights, the following algorithm will compute the entropy.

In [2]:
"""
Compute the entropy (in bits) of the given distribution, where
the elements of the distribution are relative weights.  The supplied
weights should be non-negative values, but no checking is done.  Not
all weights can be zero, or an error will occur.

The entropy is computed in *bits*, or using base 2 logarithms.  So a
choice among 4 different options of equal weight will have entropy of
2 bits.

# Examples
```julia
julia> h = get_entropy([1,2,7,9])
```
"""
function get_entropy(weights::Array)
    # Get the sum of all the weights.
    T = sum(weights)
    # Compute the entropy.
    return foldl((h,w) -> h = h - w/T * log(w/T) / log(2.0), weights; init=0.0)
end

get_entropy

Assume that $k$ of the weights are known, with $1\le k< n$, and that the remaining $n-k$ values are unknown.  Without loss of generality, assume the known values are the first $k$; $w_1, w_2, \ldots, w_k$, with subsequent weights unknown.  We can rewrite the equation for entropy as follows, dividing the known and unknown weights.  Note that now $T$ is itself an unknown!

$$
H[W] =
- \overbrace{\sum_{i=1}^k \frac{w_i}{T} \ln \frac{w_i}{T}}^{\mbox{known}}
- \overbrace{\sum_{i=k+1}^n \frac{w_i}{T} \ln \frac{w_i}{T}}^{\mbox{unknown}}
$$

We want to choose values for the unknown weights so that the overall entropy is maximized.  Since entropy is maximized for a uniform distribution, we must choose the same value $u$ for every unknown weight.  Replacing the unknown weights with $u$ results in the following equation.

$$
\begin{align*}
H[W] & = - \sum_{i=1}^k \frac{w_i}{T} \ln \frac{w_i}{T} - \sum_{i=k+1}^n \frac{u}{T} \ln \frac{u}{T} \\
     & = - \sum_{i=1}^k \frac{w_i}{T} \ln \frac{w_i}{T} - (n-k)\frac{u}{T} \ln \frac{u}{T} \\
\end{align*}
$$

Let the sum of the *known* values be denoted $K = \sum_{i=1}^k w_i$, and let the "partial entropy" of the knowns be $E = \sum_{i=1}^k w_i \ln w_i$.  We substitute these into the entropy equation.

$$
\begin{align*}
H[W] & = - \sum_{i=1}^k \frac{w_i}{T} \ln \frac{w_i}{T} - (n-k)\frac{u}{T} \ln \frac{u}{T} \\
     & = - \frac{1}{T} \left[ \sum_{i=1}^k w_i \ln \frac{w_i}{T} + (n-k)u \ln \frac{u}{T} \right] \\
     & = - \frac{1}{T} \left[ \sum_{i=1}^k w_i (\ln w_i - \ln T) + (n-k)u (\ln u - \ln T) \right] \\
     & = - \frac{1}{T} \left[ \sum_{i=1}^k w_i \ln w_i - \sum_{i=1}^k w_i \ln T + (n-k)u (\ln u - \ln T) \right] \\
     & = - \frac{1}{T} \left[ E - \ln T \sum_{i=1}^k w_i + (n-k)u (\ln u - \ln T) \right] \\
     & = - \frac{1}{T} \left[ E - K \ln T + (n-k)u (\ln u - \ln T) \right] \\
\end{align*}
$$

We now have an equation with two unknowns: $T$ and $u$.  Note that the value of the unknown weight $u$ can be written in terms of the unknown sum $T$ as follows.

$$ u = \frac{T-K}{n-k} $$

We substitute this into the entropy equation.

$$
\begin{align*}
H[W] & = - \frac{1}{T} \left[ E - K \ln T + (n-k)u (\ln u - \ln T) \right] \\
     & = - \frac{1}{T} \left[ E - K \ln T + (n-k) \frac{T-K}{n-k} (\ln \frac{T-K}{n-k} - \ln T) \right] \\
     & = - \frac{1}{T} \left[ E - K \ln T + (T-K) (\ln \frac{T-K}{n-k} - \ln T) \right] \\
     & = - \frac{1}{T} \left[ E - K \ln T + (T-K) \ln \frac{T-K}{n-k} - (T-K) \ln T \right] \\
     & = - \frac{1}{T} \left[ E - K \ln T + (T-K) \ln \frac{T-K}{n-k} - T\ln T + K\ln T \right] \\
     & = - \frac{1}{T} \left[ E + (T-K) \ln \frac{T-K}{n-k} - T\ln T \right] \\
     & = - \frac{E}{T} - \frac{(T-K)}{T} \ln \frac{T-K}{n-k} + \ln T \\
\end{align*}
$$

Now we have an expression for the entropy in terms of the single unknown $T$.  We need to find the value of $T$ that maximizes this equation.  To find that, we will set the derivative equal to zero.

We compute the derivative of each of the three terms with respect to $T$ as follows.

The first term:

$$
\begin{align*}
\frac{d}{dT} \left[ - \frac{E}{T} \right] & = -E \frac{d}{dT} \frac{1}{T} = \frac{E}{T^2} \\
\end{align*}
$$

The second term:

$$
\begin{align*}
\frac{d}{dT} \left[ - \frac{(T-K)}{T} \ln \frac{T-K}{n-k} \right] & = - \frac{(T-K)}{T} \left[ \frac{d}{dT} \ln \frac{T-K}{n-k} \right] - \ln \frac{T-K}{n-k} \left[ \frac{d}{dT} \frac{(T-K)}{T} \right] \\
    & = - \frac{(T-K)}{T} \left[ \frac{n-k}{T-K} \frac{1}{n-k} \right] - \ln \frac{T-K}{n-k} \left[ \frac{d}{dT} \left( 1-\frac{K}{T} \right) \right] \\
    & = - \frac{1}{T} - \ln \frac{T-K}{n-k} \left[ \frac{K}{T^2} \right] \\
\end{align*}
$$

The third term:

$$
\begin{align*}
\frac{d}{dT} \left[ \ln T \right] & = \frac{d}{dT} \ln T = \frac{1}{T} \\
\end{align*}
$$

The complete derivative is the sum of these three terms.

$$
\begin{align*}
\frac{d}{dT} H[W] & = \frac{E}{T^2} - \frac{1}{T} - \ln \frac{T-K}{n-k} \left[ \frac{K}{T^2} \right] + \frac{1}{T} \\
    & = \frac{E}{T^2} - \ln \frac{T-K}{n-k} \left[ \frac{K}{T^2} \right] \\
\end{align*}
$$

This is zero when the numerator is zero, so we write the following.

$$
\begin{align*}
0 & = E - K \ln \frac{T-K}{n-k} \\
E  & = K \ln \frac{T-K}{n-k} \\
\frac{E}{K}  & = \ln \frac{T-K}{n-k} \\
\frac{E}{K}  & = \ln u \\
e^\frac{E}{K}  & = u \\
\end{align*}
$$

The result gives a value for the unknown weight $u$ that is *independent of the number of unknowns*.  The following function computes the value of $u$ given the set of known weights.

Note that if $E$ is computed using logarithms of base $b$, then the result will actually be $b^{E/K}$, instead.  Note also that the base will *make no difference in the result*; one must simply be consistent.

In [5]:
"""
Compute a weight that maximizes the entropy of the
distribution given the other, known weights.

# Examples
```julia
julia> u = get_emax_weight([1,2,7,9])
```
"""
function get_emax_weight(weights::Array)
    # Compute the sum of the weights.
    K = sum(weights)
    # Compute the partial entropy of the weights.
    E = foldl((h,w) -> h = h + w * log(w), weights; init=0.0)
    # Return the value to use to maximize the single-step entropy.
    return exp(E/K)
end

get_emax_weight